In [95]:
import numpy as np
import pandas as pd

In [3]:
# Load the GTF file into a DataFrame
gtf_file = "../ignoreme-tools/data/output.gtf"
gtf_columns = ["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"]
gtf_df = pd.read_csv(gtf_file, sep="\t", comment='#', names=gtf_columns)

/var/folders/s8/g_638jvd1pq8yhzr9r71mx3c0000gn/T/ipykernel_13694/833650171.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf_df = pd.read_csv(gtf_file, sep="\t", comment='#', names=gtf_columns)


In [125]:
# helper to parse attributes and add them as a new column
def extract_attribute(df, attr):
    def _parse_attribute(col, attr):
        # (some attributes are optional, that's why we need NaN)
        try: value = col.split(f'{attr} "')[1].split('";')[0]
        except: return np.NaN
    
        try: return float(value)
        except: return value
    df[attr] = df['attribute'].apply(_parse_attribute, args=(attr, ))

In [161]:
# get transcripts
transcripts = gtf_df[gtf_df['feature'] == 'transcript'].loc[:]

# parse attributes
for attr in ['gene_id', 'transcript_id', 'FPKM']:
    extract_attribute(transcripts, attr)
del transcripts['attribute']

# get total FPKM score of gene (we would get the same result with TPM)
transcripts['FPKM_gene_total'] = transcripts.groupby('gene_id')['FPKM'].transform('sum')
# get relative expression for transcript, i.e. what is the probability of this transcript given the gene is transcribed?
transcripts['transcript_expression'] = transcripts['FPKM'] / transcripts['FPKM_gene_total']

transcripts

,seqname,source,feature,start,end,score,strand,frame,gene_id,transcript_id,FPKM,FPKM_gene_total,transcript_expression
0,1,StringTie,transcript,14431,29392,1000,-,.,STRG.1,STRG.1.1,4.588552,11.521000,0.398277
10,1,StringTie,transcript,14431,29392,1000,-,.,STRG.1,STRG.1.2,2.204759,11.521000,0.191369
21,1,StringTie,transcript,14431,29392,1000,-,.,STRG.1,STRG.1.3,4.156013,11.521000,0.360734
31,1,StringTie,transcript,14431,29392,1000,-,.,STRG.1,STRG.1.4,0.571676,11.521000,0.049620
40,1,StringTie,transcript,91551,112746,1000,-,.,STRG.2,STRG.2.1,0.672025,0.672025,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
372614,KI270745.1,StringTie,transcript,13580,16138,1000,+,.,STRG.19755,STRG.19755.1,2.617983,2.617983,1.000000
372618,KI270745.1,StringTie,transcript,19205,21444,1000,+,.,STRG.19756,STRG.19756.1,3.496349,3.496349,1.000000
372621,KI270745.1,StringTie,transcript,21817,22936,1000,+,.,STRG.19757,STRG.19757.1,1.570031,2.145621,0.731737
372624,KI270745.1,StringTie,transcript,22234,22936,1000,+,.,STRG.19757,STRG.19757.2,0.575590,2.145621,0.268263


In [165]:
# get exons
exons = gtf_df[gtf_df['feature'] == 'exon'].loc[:]

# parse attributes
extract_attribute(exons, 'transcript_id')
del exons['attribute']

# merge relative expression of transcript into exons
exons = pd.merge(exons, transcripts[['transcript_id', 'transcript_expression']], on='transcript_id')
# sum up relative expressions of transcripts to receive donor/acceptor probabilities
p_acceptor = exons.groupby('start')['transcript_expression'].sum().rename('p_acceptor')
p_donor = exons.groupby('end')['transcript_expression'].sum().rename('p_donor')

p_acceptor, p_donor

(start
 577          1.000000
 648          1.000000
 1602         1.000000
 1671         1.000000
 2585         0.633422
                ...   
 248858918    0.333762
 248906196    1.000000
 248913816    1.000000
 248917279    1.000000
 248936581    1.000000
 Name: p_acceptor, Length: 171820, dtype: float64,
 end
 647          1.000000
 1601         1.000000
 1670         1.000000
 2692         1.000000
 3229         1.000000
                ...   
 248859085    0.175148
 248906342    1.000000
 248913879    1.000000
 248919946    1.000000
 248937043    1.000000
 Name: p_donor, Length: 171666, dtype: float64)